<a href="https://colab.research.google.com/github/gvanathip/classification/blob/main/Random_Tunning_LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset and Clean Process : https://www.kaggle.com/code/iyet1killer/classification-on-categorical-data-part-1-sklearn/notebook

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('in-vehicle-coupon-recommendation.csv')

In [3]:
df.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0


In [4]:
# Ultimate Data Processing Function
def data_cleaning(data=df):
    clean_df = df.copy()

    clean_df.drop(columns=['car'], inplace=True)
    na_columns = ['Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20', 'Restaurant20To50']
    clean_df.drop(columns=['toCoupon_GEQ5min'], inplace=True)
    frequency_map = {
        'never': 0,
        'less1': 1,
        '1~3': 2,
        '4~8': 3,
        'gt8': 4}
    age_map = {
        'below21': 0,
        '21': 1,
        '26': 2,
        '31': 3,
        '36': 4,
        '41': 5,
        '46': 6,
        '50plus': 7}
    income_map = {
        'Less than $12500': 0,
        '$12500 - $24999': 1,
        '$25000 - $37499': 2,
        '$37500 - $49999': 3,
        '$50000 - $62499': 4,
        '$62500 - $74999': 5,
        '$75000 - $87499': 6,
        '$87500 - $99999': 7,
        '$100000 or More': 8}
    frequency_cols = ['Restaurant20To50', 'RestaurantLessThan20',
                      'CarryAway', 'CoffeeHouse', 'Bar']
    for col in frequency_cols:
        clean_df[col] = clean_df[col].map(frequency_map)
    clean_df.age = clean_df.age.map(age_map)
    clean_df.income = clean_df.income.map(income_map)
    clean_df.drop(columns=['direction_opp'], inplace=True)
    clean_df['distance'] = None
    clean_df.loc[clean_df['toCoupon_GEQ15min'] == 0, 'distance'] = 0
    clean_df.loc[(clean_df['toCoupon_GEQ15min'] == 1) & \
                 (clean_df['toCoupon_GEQ25min'] == 0), 'distance'] = 1
    clean_df.loc[clean_df['toCoupon_GEQ25min'] == 1, 'distance'] = 2
    clean_df.distance = clean_df.distance.astype('int64')
    clean_df.drop(columns=['toCoupon_GEQ15min', 'toCoupon_GEQ25min'], inplace=True)
    clean_df.has_children = clean_df.has_children.astype(str)
    clean_df.direction_same = clean_df.direction_same.astype(str)
    return clean_df

In [5]:
df = data_cleaning(df)

In [6]:
df.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,occupation,income,Bar,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,direction_same,Y,distance
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,1,Unmarried partner,...,Unemployed,3,0.0,0.0,NaN,3.0,2.0,0,1,0
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,1,Unmarried partner,...,Unemployed,3,0.0,0.0,NaN,3.0,2.0,0,0,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,1,Unmarried partner,...,Unemployed,3,0.0,0.0,NaN,3.0,2.0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,1,Unmarried partner,...,Unemployed,3,0.0,0.0,NaN,3.0,2.0,0,0,1
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,1,Unmarried partner,...,Unemployed,3,0.0,0.0,NaN,3.0,2.0,0,0,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12684 entries, 0 to 12683
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   destination           12684 non-null  object 
 1   passanger             12684 non-null  object 
 2   weather               12684 non-null  object 
 3   temperature           12684 non-null  int64  
 4   time                  12684 non-null  object 
 5   coupon                12684 non-null  object 
 6   expiration            12684 non-null  object 
 7   gender                12684 non-null  object 
 8   age                   12684 non-null  int64  
 9   maritalStatus         12684 non-null  object 
 10  has_children          12684 non-null  object 
 11  education             12684 non-null  object 
 12  occupation            12684 non-null  object 
 13  income                12684 non-null  int64  
 14  Bar                   12577 non-null  float64
 15  CoffeeHouse        

In [8]:
df.groupby(['Y'])['Y'].count()

Y
0    5474
1    7210
Name: Y, dtype: int64

In [9]:
df.isnull().sum()

destination               0
passanger                 0
weather                   0
temperature               0
time                      0
coupon                    0
expiration                0
gender                    0
age                       0
maritalStatus             0
has_children              0
education                 0
occupation                0
income                    0
Bar                     107
CoffeeHouse             217
CarryAway               151
RestaurantLessThan20    130
Restaurant20To50        189
direction_same            0
Y                         0
distance                  0
dtype: int64

In [10]:
num_cols = sorted(list(df.select_dtypes(include="number").columns))
cate_cols = sorted(list(df.select_dtypes(exclude="number").columns))

In [11]:
df[num_cols] = df[num_cols].fillna(0)
df[cate_cols] = df[cate_cols].fillna('-')

In [12]:
df.isnull().sum()

destination             0
passanger               0
weather                 0
temperature             0
time                    0
coupon                  0
expiration              0
gender                  0
age                     0
maritalStatus           0
has_children            0
education               0
occupation              0
income                  0
Bar                     0
CoffeeHouse             0
CarryAway               0
RestaurantLessThan20    0
Restaurant20To50        0
direction_same          0
Y                       0
distance                0
dtype: int64

In [13]:
X = df.drop("Y", axis=1)
y = df.Y

In [14]:
import numpy as np

In [15]:
X_num = np.array(X.select_dtypes(include="number"))

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
standard_scaler = StandardScaler()

In [18]:
standard_scaled_data = standard_scaler.fit_transform(X_num)

In [19]:
X_num.shape

(12684, 9)

In [20]:
standard_scaled_data.shape

(12684, 9)

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split

In [22]:
enc = OneHotEncoder(handle_unknown = 'ignore', sparse=False)

In [23]:
enc.fit(X.select_dtypes(exclude="number").astype(str))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


OneHotEncoder(handle_unknown='ignore', sparse=False, sparse_output=False)

In [24]:
encoded_columns = enc.get_feature_names_out(input_features=pd.DataFrame(X.select_dtypes(exclude="number").astype(str)).columns)

In [25]:
encoded_columns

array(['destination_Home', 'destination_No Urgent Place',
       'destination_Work', 'passanger_Alone', 'passanger_Friend(s)',
       'passanger_Kid(s)', 'passanger_Partner', 'weather_Rainy',
       'weather_Snowy', 'weather_Sunny', 'time_10AM', 'time_10PM',
       'time_2PM', 'time_6PM', 'time_7AM', 'coupon_Bar',
       'coupon_Carry out & Take away', 'coupon_Coffee House',
       'coupon_Restaurant(20-50)', 'coupon_Restaurant(<20)',
       'expiration_1d', 'expiration_2h', 'gender_Female', 'gender_Male',
       'maritalStatus_Divorced', 'maritalStatus_Married partner',
       'maritalStatus_Single', 'maritalStatus_Unmarried partner',
       'maritalStatus_Widowed', 'has_children_0', 'has_children_1',
       'education_Associates degree', 'education_Bachelors degree',
       'education_Graduate degree (Masters or Doctorate)',
       'education_High School Graduate', 'education_Some High School',
       'education_Some college - no degree',
       'occupation_Architecture & Engineering

In [26]:
X_cat = enc.transform(X.select_dtypes(exclude="number"))

In [27]:
num_feature_names = X.select_dtypes(include="number").columns

In [28]:
cat_feature_names = enc.get_feature_names_out(X.select_dtypes(exclude="number").columns)

In [29]:
feature_names = list(num_feature_names)+list(cat_feature_names)

In [30]:
feature_names

['temperature',
 'age',
 'income',
 'Bar',
 'CoffeeHouse',
 'CarryAway',
 'RestaurantLessThan20',
 'Restaurant20To50',
 'distance',
 'destination_Home',
 'destination_No Urgent Place',
 'destination_Work',
 'passanger_Alone',
 'passanger_Friend(s)',
 'passanger_Kid(s)',
 'passanger_Partner',
 'weather_Rainy',
 'weather_Snowy',
 'weather_Sunny',
 'time_10AM',
 'time_10PM',
 'time_2PM',
 'time_6PM',
 'time_7AM',
 'coupon_Bar',
 'coupon_Carry out & Take away',
 'coupon_Coffee House',
 'coupon_Restaurant(20-50)',
 'coupon_Restaurant(<20)',
 'expiration_1d',
 'expiration_2h',
 'gender_Female',
 'gender_Male',
 'maritalStatus_Divorced',
 'maritalStatus_Married partner',
 'maritalStatus_Single',
 'maritalStatus_Unmarried partner',
 'maritalStatus_Widowed',
 'has_children_0',
 'has_children_1',
 'education_Associates degree',
 'education_Bachelors degree',
 'education_Graduate degree (Masters or Doctorate)',
 'education_High School Graduate',
 'education_Some High School',
 'education_Some col

In [31]:
X_processed = np.concatenate((standard_scaled_data, X_cat), axis=1)

In [32]:
y_processed = y.values.reshape(-1, 1)
# y_processed[y_processed=='redeem lotus'] = 1
# y_processed[y_processed=='not redeem lotus'] = 0
y_processed = np.ravel(y_processed.astype(int))

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y_processed, stratify=y_processed, random_state=1121218, test_size = 0.3
)

In [34]:
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

In [35]:
import time
from datetime import datetime
def evaluate(model, X, y, cv):
    # models_scores_table = pd.DataFrame()


    # for m in models:
    # print(m)
    start_time = time.time()
    scoring = {'accuracy':make_scorer(accuracy_score),
           'precision':make_scorer(precision_score),
           'recall':make_scorer(recall_score),
           'f1_score':make_scorer(f1_score),
           'AUC':make_scorer(roc_auc_score)}
    cv = cross_validate(model, X, y, cv=cv, scoring=scoring, return_train_score=False)
    end_time = time.time()
    fit_time = end_time - start_time
    cv_df = pd.DataFrame({'LGBM':[cv['test_accuracy'].mean(),
                             cv['test_precision'].mean(),
                             cv['test_recall'].mean(),
                             cv['test_f1_score'].mean(),
                             cv['test_AUC'].mean(),
                             fit_time]},index=['Accuracy', 'Precision', 'Recall', 'F1 Score','AUC','fit_time'])
#     print(cv_df.T)

    # models_scores_table = pd.concat([models_scores_table,cv_df.T])
    #     print(models_scores_table)
    # print('done')

    cv_df = cv_df.T

    return cv['test_accuracy'].mean(), cv['test_precision'].mean(), cv['test_recall'].mean(), cv['test_f1_score'].mean(), cv['test_AUC'].mean(), fit_time
#     return cv_df[measurement].values[0]

In [36]:
from datetime import datetime

current_dateTime = datetime.now()

print(current_dateTime)

2023-12-07 06:38:39.075078


In [37]:
default_clf = LGBMClassifier()

In [38]:
_ = default_clf.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 5047, number of negative: 3831
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001442 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 185
[LightGBM] [Info] Number of data points in the train set: 8878, number of used features: 73
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.568484 -> initscore=0.275668
[LightGBM] [Info] Start training from score 0.275668


In [39]:
d_accuracy, d_precision, d_recall, d_f1, d_auc, d_fittime = evaluate(default_clf, X_test, y_test, cv = 3)

[LightGBM] [Info] Number of positive: 1442, number of negative: 1095
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000431 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 179
[LightGBM] [Info] Number of data points in the train set: 2537, number of used features: 70
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.568388 -> initscore=0.275277
[LightGBM] [Info] Start training from score 0.275277
[LightGBM] [Info] Number of positive: 1442, number of negative: 1095
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000399 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 179
[LightGBM] [Info] Number of data points in the train set: 2537, number of used features: 70
[LightGBM] [Info] [binary:Bo

In [40]:
d_auc

0.7112743854291056

In [41]:
default_clf.get_params(deep=True)

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [42]:
d_accuracy, d_precision, d_recall, d_f1, d_auc, d_fittime = evaluate(default_clf, X_test, y_test, cv = 3)

[LightGBM] [Info] Number of positive: 1442, number of negative: 1095
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000533 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 179
[LightGBM] [Info] Number of data points in the train set: 2537, number of used features: 70
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.568388 -> initscore=0.275277
[LightGBM] [Info] Start training from score 0.275277
[LightGBM] [Info] Number of positive: 1442, number of negative: 1095
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000427 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 179
[LightGBM] [Info] Number of data points in the train set: 2537, number of used features: 70
[LightGBM] [Info] [binary:Bo

In [45]:
import pickle
with open('trained_model_LGBM_default.pkl', 'wb') as file:
    pickle.dump(default_clf, file)

In [47]:
feature_importance_default = default_clf.feature_importances_

In [49]:
feature_importance_default = 100.0 * (feature_importance_default / feature_importance_default.max())
feature_importance_default_df = pd.DataFrame({"features":feature_names}).join(pd.DataFrame({"Importance":feature_importance_default})).set_index('features')
feature_importance_default_df = feature_importance_default_df.reset_index().sort_values(by='Importance', ascending=False)
feature_importance_default_df.to_csv("features_importance_default.csv",index=False)

In [50]:
import warnings

# Suppress FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)

classlightgbm.LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=None, importance_type='split', **kwargs)

In [51]:
d_auc

0.7112743854291056

In [52]:
# threshole = d_auc

In [53]:
# threshole

In [54]:
# best_auc = threshole

In [55]:
log_file = 'fine_tuning_log.csv'

In [56]:
import os.path
os.path.isfile(log_file)

False

In [ ]:
df_record = pd.DataFrame(columns=[
                              'learning_rate',
                              'max_depth',
                              'n_estimators',
                              'num_leaves',
                              'reg_alpha',
                              'reg_lambda',
                              'subsample_for_bin',
                              'min_split_gain',
                              'min_child_weight',
                              'min_child_samples',
                              'subsample',
                              'colsample_bytree',
#                               'feature_fraction',
#                               'bagging_fraction',
                              'accuracy','precision','recall','f1','auc','fittime',
                             'time_stamp'])

a = default_clf.get_params(deep=True)['learning_rate']
b = default_clf.get_params(deep=True)['max_depth']
c = default_clf.get_params(deep=True)['n_estimators']
d = default_clf.get_params(deep=True)['num_leaves']
e = default_clf.get_params(deep=True)['reg_alpha']
f = default_clf.get_params(deep=True)['reg_lambda']
g = default_clf.get_params(deep=True)['subsample_for_bin']
h = default_clf.get_params(deep=True)['min_split_gain']
j = default_clf.get_params(deep=True)['min_child_weight']
k = default_clf.get_params(deep=True)['min_child_samples']
l = default_clf.get_params(deep=True)['subsample']
m = default_clf.get_params(deep=True)['colsample_bytree']
# p = 1.0
# q = 1.0

# clf = LGBMClassifier(objective = 'binary',
#                      learning_rate = a,
#                      max_depth = b,
#                      n_estimators = c,
#                      num_leaves = d,
#                      reg_alpha=e,
#                      reg_lambda=f,
#                      subsample_for_bin=g,
#                      min_split_gain=h,
#                      min_child_weight=j,
#                      min_child_samples=k,
#                      subsample=l,
#                      colsample_bytree=m,
#                      feature_fraction=p,
#                      bagging_fraction=q,
#                      verbosity=-1,
#                      silent=True)

# _ = clf.fit(X_train, y_train)
accuracy, precision, recall, f1, auc, fittime = evaluate(default_clf, X_test, y_test, cv = 3)

record = ({
    'learning_rate' :a,
    'max_depth':b,
    'n_estimators':c,
    'num_leaves':d,
    'reg_alpha':e,
    'reg_lambda':f,
    'subsample_for_bin':g,
    'min_split_gain':h,
    'min_child_weight':j,
    'min_child_samples':k,
    'subsample':l,
    'colsample_bytree':m,
#     'feature_fraction':p,
#     'bagging_fraction':q,
    'accuracy':accuracy,
    'precision':precision,
    'recall':recall,
    'f1':f1,
    'auc':auc,
    'fittime':fittime,
    'time_stamp':datetime.now()
               })
df_record = df_record.append(record, ignore_index=True)
df_record = df_record.sort_values(by=['auc'], ascending=False)
df_record.to_csv(log_file,index=False)

In [ ]:
df_record_exist = pd.read_csv(log_file)
df_record_exist

,learning_rate,max_depth,n_estimators,num_leaves,reg_alpha,reg_lambda,subsample_for_bin,min_split_gain,min_child_weight,min_child_samples,...,colsample_bytree,feature_fraction,bagging_fraction,accuracy,precision,recall,f1,auc,fittime,time_stamp
0,0.1,-1,100,31,0.0,0.0,200000,0.0,0.001,20,...,1.0,1.0,1.0,0.721226,0.740623,0.784096,0.761729,0.711274,0.36039,2023-12-01 08:09:22.321315


In [ ]:
threshole = 0.8 ###target the loop will stop tunning when AUC achieve threshole

In [ ]:
def save_tunning():

    print('saving new best model')
    df_record = pd.read_csv(log_file)
    df_record = df_record.sort_values(by=['auc'], ascending=False)
    best_clf = LGBMClassifier(objective = 'binary',
                      learning_rate = df_record.head(1).learning_rate.values[0],
                      max_depth = df_record.head(1).max_depth.values[0],
                      n_estimators = df_record.head(1).n_estimators.values[0],
                      num_leaves = df_record.head(1).num_leaves.values[0],
                      reg_alpha=df_record.head(1).reg_alpha.values[0],
                      reg_lambda=df_record.head(1).reg_lambda.values[0],
                      subsample_for_bin=df_record.head(1).subsample_for_bin.values[0],
                      min_split_gain=df_record.head(1).min_split_gain.values[0],
                      min_child_weight=df_record.head(1).min_child_weight.values[0],
                      min_child_samples=df_record.head(1).min_child_samples.values[0],
                      subsample=df_record.head(1).subsample.values[0],
                      colsample_bytree=df_record.head(1).colsample_bytree.values[0],
#                       feature_fraction=df_record.head(1).feature_fraction.values[0],
#                       bagging_fraction=df_record.head(1).bagging_fraction.values[0],
                      verbosity=-1,
                      silent=True)

    _ = best_clf.fit(X_train, y_train)
    best_accuracy, best_precision, best_recall, best_f1, best_auc, best_fittime = evaluate(best_clf, X_test, y_test, cv = 3)

    with open('trained_model_LGBM_tunning.pkl', 'wb') as file:
        pickle.dump(best_clf, file)

    feature_importance_tunning = best_clf.feature_importances_
    feature_importance_tunning = 100.0 * (feature_importance_tunning / feature_importance_tunning.max())
    feature_importance_tunning_df = pd.DataFrame({"features":feature_names}).join(pd.DataFrame({"Importance":feature_importance_tunning})).set_index('features')
    feature_importance_tunning_df = feature_importance_tunning_df.reset_index().sort_values(by='Importance', ascending=False)
    feature_importance_tunning_df.to_csv("features_importance_tunning.csv",index=False)

    print('saved new best model')
    print('new best auc :',best_auc)

    return best_auc


In [ ]:
best_auc = save_tunning()

In [ ]:
best_auc

0.7112743854291056

In [ ]:
def reset_param():

    print('reseting parameter')
    learning_rate = list(np.round(np.arange(1.00, 0.001, -0.001),3))
    max_depth = list(np.arange(1,15,1))
    n_estimators = list(np.arange(1,2000,1))
    num_leaves = list(np.arange(1,2000,1))
    reg_alpha = list(np.round(np.arange(0.00,1.00,0.001),3))
    reg_lambda = list(np.round(np.arange(0.00,1.00,0.001),3))

    subsample_for_bin = list(np.arange(100000, 1000000, 1000))
    min_split_gain = list(np.round(np.arange(0.0, 1.0, 0.001),3))
    min_child_weight = list(np.round(np.arange(0.000, 1.000, 0.001),3))
    min_child_samples = list(np.arange(0, 1000, 1))
    subsample = list(np.round(np.arange(0.0, 1.0, 0.001),3))
    colsample_bytree = list(np.round(np.arange(0.0, 1.0, 0.001),3))
#     feature_fraction = list(np.round(np.arange(0.0, 1.0, 0.001),3))
#     bagging_fraction = list(np.round(np.arange(0.0, 1.0, 0.001),3))

#     print('reset parameter')
    print('learning rate =',learning_rate[0],'-',learning_rate[-1],'\t\t range =',len(learning_rate))
    print('max depth =',max_depth[0],'-',max_depth[-1],'\t\t\t range =',len(max_depth))
    print('n_estimators =',n_estimators[0],'-',n_estimators[-1],'\t\t range =',len(n_estimators))
    print('num leaves =',num_leaves[0],'-',num_leaves[-1],'\t\t\t range =',len(num_leaves))
    print('reg alpha =',reg_alpha[0],'-',reg_alpha[-1],'\t\t range =',len(reg_alpha))
    print('reg lambda =',reg_lambda[0],'-',reg_lambda[-1],'\t\t range =',len(reg_lambda))
    print('subsample for bin =',subsample_for_bin[0],'-',subsample_for_bin[-1],'\t range =',len(subsample_for_bin))
    print('min split gain =',min_split_gain[0],'-',min_split_gain[-1],'\t\t range =',len(min_split_gain))
    print('min child weight =',min_child_weight[0],'-',min_child_weight[-1],'\t\t range =',len(min_child_weight))
    print('min child samples =',min_child_samples[0],'-',min_child_samples[-1],'\t\t range =',len(min_child_samples))
    print('subsample =',subsample[0],'-',subsample[-1],'\t\t range =',len(subsample))
    print('colsample_bytree =',colsample_bytree[0],'-',colsample_bytree[-1],'\t\t range =',len(colsample_bytree))

    return learning_rate, max_depth, n_estimators, num_leaves, reg_alpha, reg_lambda, subsample_for_bin, \
            min_split_gain, min_child_weight, min_child_samples, subsample, colsample_bytree

In [ ]:
learning_rate, max_depth, n_estimators, num_leaves, reg_alpha, reg_lambda, subsample_for_bin, \
            min_split_gain, min_child_weight, min_child_samples, subsample, colsample_bytree = reset_param()

In [ ]:
from tqdm import tqdm
import random



#reset
#-------------------------------------------------------------
# reset_param()
#-------------------------------------------------------------

n = 10  #retrain iters
top = 3  #window scope top
# r = 5 #round


# for z in range(r):
n_pass = 0
while threshole >= best_auc:
    print('-----------------------------------------------')
    # print('round :',z)

    df_record_exist = pd.read_csv(log_file)

    if (len(df_record_exist) >= top) and (n_pass < 10):
        # df_record_exist = pd.read_csv(log_file)
        print('trim parameter')

        print('min auc',df_record_exist.head(top).auc.min())
        print('max auc',df_record_exist.head(top).auc.max())

#         #comment when reset
#         #---------------------------------------
        learning_rate = list(np.round(np.arange(df_record_exist.head(top).learning_rate.max(), df_record_exist.head(top).learning_rate.min()-0.001, -0.001),3))
        max_depth = list(np.arange(df_record_exist.head(top).max_depth.min(),df_record_exist.head(top).max_depth.max()+1,1))
        n_estimators = list(np.arange(df_record_exist.head(top).n_estimators.min(),df_record_exist.head(top).n_estimators.max()+1,1))
        num_leaves = list(np.arange(df_record_exist.head(top).num_leaves.min(),df_record_exist.head(top).num_leaves.max()+1,1))
        reg_alpha = list(np.round(np.arange(df_record_exist.head(top).reg_alpha.min(),df_record_exist.head(top).reg_alpha.max()+0.01,0.001),3))
        reg_lambda = list(np.round(np.arange(df_record_exist.head(top).reg_lambda.min(),df_record_exist.head(top).reg_lambda.max()+0.01,0.001),3))

        subsample_for_bin = list(np.arange(df_record_exist.head(top).subsample_for_bin.min(), df_record_exist.head(top).subsample_for_bin.max()+1000, 1000))
        min_split_gain = list(np.round(np.arange(df_record_exist.head(top).min_split_gain.min(), df_record_exist.head(top).min_split_gain.max()+0.1, 0.001),3))
        min_child_weight = list(np.round(np.arange(df_record_exist.head(top).min_child_weight.min(), df_record_exist.head(top).min_child_weight.max()+0.001, 0.001),3))
        min_child_samples = list(np.arange(df_record_exist.head(top).min_child_samples.min(), df_record_exist.head(top).min_child_samples.max()+1, 1))
        subsample = list(np.round(np.arange(df_record_exist.head(top).subsample.min(), df_record_exist.head(top).subsample.max()+0.1, 0.001),3))
        colsample_bytree = list(np.round(np.arange(df_record_exist.head(top).colsample_bytree.min(), df_record_exist.head(top).colsample_bytree.max()+0.1, 0.001),3))
#         feature_fraction = list(np.round(np.arange(df_record_exist.head(top).feature_fraction.min(), df_record_exist.head(top).feature_fraction.max()+0.1, 0.001),3))
#         bagging_fraction = list(np.round(np.arange(df_record_exist.head(top).bagging_fraction.min(), df_record_exist.head(top).bagging_fraction.max()+0.1, 0.001),3))
#         #---------------------------------------

    else:

        print('reset parameter')
        n_pass = 0

        reset_param()





    for i in tqdm(range(n)):


        a = random.choice(learning_rate)
        b = random.choice(max_depth)
        c = random.choice(n_estimators)


        d = random.choice([x for x in num_leaves if 2^b <= x and x > 1])


        e = random.choice(reg_alpha)
        f = random.choice(reg_lambda)

        g = random.choice(subsample_for_bin)
        h = random.choice(min_split_gain)
        j = random.choice(min_child_weight)
        k = random.choice(min_child_samples)
        l = random.choice(subsample)
        m = random.choice(colsample_bytree)
#         p = random.choice(feature_fraction)
#         q = random.choice(bagging_fraction)



        clf = LGBMClassifier(objective = 'binary',
                             learning_rate = a,
                             max_depth = b,
                             n_estimators = c,
                             num_leaves = d,
                             reg_alpha=e,
                             reg_lambda=f,
                             subsample_for_bin=g,
                             min_split_gain=h,
                             min_child_weight=j,
                             min_child_samples=k,
                             subsample=l,
                             colsample_bytree=m,
#                              feature_fraction=p,
#                              bagging_fraction=q,
                             verbosity=-1,
                             silent=True)

        try:
          _ = clf.fit(X_train, y_train)
          accuracy, precision, recall, f1, auc, fittime = evaluate(clf, X_test, y_test, cv = 3)

          df_record_new = pd.DataFrame(columns=[
                            'learning_rate',
                            'max_depth',
                            'n_estimators',
                            'num_leaves',
                            'reg_alpha',
                            'reg_lambda',
                            'subsample_for_bin',
                            'min_split_gain',
                            'min_child_weight',
                            'min_child_samples',
                            'subsample',
                            'colsample_bytree',
#                             'feature_fraction',
#                             'bagging_fraction',
                            'accuracy','precision','recall','f1','auc','fittime',
                            'time_stamp'])

          record = ({
              'learning_rate' :a,
              'max_depth':b,
              'n_estimators':c,
              'num_leaves':d,
              'reg_alpha':e,
              'reg_lambda':f,
              'subsample_for_bin':g,
              'min_split_gain':h,
              'min_child_weight':j,
              'min_child_samples':k,
              'subsample':l,
              'colsample_bytree':m,
#               'feature_fraction':p,
#               'bagging_fraction':q,
              'accuracy':accuracy,
              'precision':precision,
              'recall':recall,
              'f1':f1,
              'auc':auc,
              'fittime':fittime,
              'time_stamp':datetime.now()
                          })
          df_record_new = df_record_new.append(record, ignore_index=True)
          df_record_new = df_record_new.drop_duplicates()
          df_record_new = df_record_new.sort_values(by=['auc'], ascending=False)

        except:
          pass


    try:
        print('min top existing',df_record_exist.head(top).auc.min())
        print('max new',df_record_new.auc.max())

        df_record_exist = pd.read_csv(log_file)
        if len(df_record_exist) >= top:

          if df_record_exist.head(top).auc.min() > df_record_new.auc.max():
              print('pass')
              n_pass = n_pass+1
              pass
          else:
                df_record = df_record_exist.append(df_record_new, ignore_index=True)
                df_record = df_record.drop_duplicates()
                df_record = df_record.sort_values(by=['auc'], ascending=False)
                print(df_record.head(top))
                df_record.to_csv(log_file,index=False)

                best_auc = save_tunning()


        else:
            df_record = df_record_exist.append(df_record_new, ignore_index=True)
            df_record = df_record.drop_duplicates()
            df_record = df_record.sort_values(by=['auc'], ascending=False)
            print(df_record.head(top))
            df_record.to_csv(log_file,index=False)

            best_auc = save_tunning()

    except:
        print('save initial')
        df_record_new = df_record_new.sort_values(by=['auc'], ascending=False)
        df_record_new = df_record_new.drop_duplicates()
        print(df_record_new.head(top))
        df_record_new.to_csv(log_file,index=False)



print('end')



-----------------------------------------------
reset parameter


100%|██████████| 100/100 [02:20<00:00,  1.40s/it]


min top existing 0.7112743854291056
max new 0.5983308549421706
   learning_rate max_depth n_estimators num_leaves  reg_alpha  reg_lambda  \
0          0.100        -1          100         31      0.000       0.000   
1          0.306         2         1987        330      0.426       0.996   

  subsample_for_bin  min_split_gain  min_child_weight min_child_samples  ...  \
0            200000           0.000             0.001                20  ...   
1            577000           0.013             0.732               987  ...   

   colsample_bytree  feature_fraction  bagging_fraction  accuracy  precision  \
0             1.000             1.000             1.000  0.721226   0.740623   
1             0.765             0.297             0.296  0.622700   0.638056   

     recall        f1       auc   fittime                  time_stamp  
0  0.784096  0.761729  0.711274  0.360390  2023-12-01 08:09:22.321315  
1  0.776699  0.700584  0.598331  0.752592  2023-12-01 08:12:43.302813  

[2 row

100%|██████████| 100/100 [02:20<00:00,  1.41s/it]


min top existing 0.5983308549421706
max new 0.6777867257756759
   learning_rate max_depth n_estimators num_leaves  reg_alpha  reg_lambda  \
0          0.100        -1          100         31      0.000       0.000   
2          0.880         9          480       1984      0.444       0.705   
1          0.306         2         1987        330      0.426       0.996   

  subsample_for_bin  min_split_gain  min_child_weight min_child_samples  ...  \
0            200000           0.000             0.001                20  ...   
2            784000           0.158             0.980               363  ...   
1            577000           0.013             0.732               987  ...   

   colsample_bytree  feature_fraction  bagging_fraction  accuracy  precision  \
0             1.000             1.000             1.000  0.721226   0.740623   
2             0.974             0.829             0.464  0.690224   0.710036   
1             0.765             0.297             0.296  0.622700  

...

100%|██████████| 100/100 [01:27<00:00,  1.15it/s]


min top existing 0.6950388172702548
max new 0.6993859598193648
     learning_rate max_depth n_estimators num_leaves  reg_alpha  reg_lambda  \
0            0.100        -1          100         31      0.000       0.000   
1            0.101         0          391        877      0.695       0.083   
2            0.498         8          921       1204      0.492       0.202   
3            0.330        12          846       1224      0.354       0.809   
4            0.127         0          919       1243      0.897       0.068   
5            0.311        -1          248        520      0.218       0.510   
6            0.486         5          153        489      0.014       0.848   
7            0.198        12          174         43      0.010       0.250   
8            0.157         5          124       1396      0.024       0.181   
9            0.256        -1         1155        696      0.496       0.075   
10           0.226        10          250        148      0.646     

100%|██████████| 100/100 [01:26<00:00,  1.16it/s]


min top existing 0.695390907458794
max new 0.7049286717752506
     learning_rate max_depth n_estimators num_leaves  reg_alpha  reg_lambda  \
0            0.100        -1          100         31      0.000       0.000   
1            0.101         0          391        877      0.695       0.083   
126          0.414         8          491       1240      0.249       0.792   
2            0.498         8          921       1204      0.492       0.202   
3            0.330        12          846       1224      0.354       0.809   
4            0.127         0          919       1243      0.897       0.068   
5            0.311        -1          248        520      0.218       0.510   
6            0.486         5          153        489      0.014       0.848   
7            0.198        12          174         43      0.010       0.250   
8            0.157         5          124       1396      0.024       0.181   
9            0.256        -1         1155        696      0.496      

 52%|█████▏    | 52/100 [00:40<00:28,  1.69it/s]

In [ ]:
df_record_exist = pd.read_csv(log_file)
len(df_record_exist)

16

In [ ]:
d_auc

0.7112743854291056

In [ ]:
best_auc

0.7123818117946903

In [ ]:
df_record = pd.read_csv('fine_tuning_log.csv')

In [ ]:
df_record = df_record.sort_values(by=['auc'], ascending=False)

In [ ]:
best_clf = LGBMClassifier(objective = 'binary',
                     learning_rate = df_record.head(1).learning_rate.values[0],
                     max_depth = df_record.head(1).max_depth.values[0],
                     n_estimators = df_record.head(1).n_estimators.values[0],
                     num_leaves = df_record.head(1).num_leaves.values[0],
                     reg_alpha=df_record.head(1).reg_alpha.values[0],
                     reg_lambda=df_record.head(1).reg_lambda.values[0],
                     subsample_for_bin=df_record.head(1).subsample_for_bin.values[0],
                     min_split_gain=df_record.head(1).min_split_gain.values[0],
                     min_child_weight=df_record.head(1).min_child_weight.values[0],
                     min_child_samples=df_record.head(1).min_child_samples.values[0],
                     subsample=df_record.head(1).subsample.values[0],
                     colsample_bytree=df_record.head(1).colsample_bytree.values[0],
                     feature_fraction=df_record.head(1).feature_fraction.values[0],
                     bagging_fraction=df_record.head(1).bagging_fraction.values[0],
                     verbosity=-1,
                     silent=True)

In [ ]:
_ = best_clf.fit(X_train, y_train)
best_accuracy, best_precision, best_recall, best_f1, best_auc, best_fittime = evaluate(best_clf, X_test, y_test, cv = 3)

In [ ]:
best_accuracy

0.7167586854366728

In [ ]:
best_precision

0.7422156869511144

In [ ]:
best_recall

0.7688395746648173

In [ ]:
best_f1

0.7552705825420319

In [ ]:
best_auc

0.708513383457257